In [3]:
import nflreadpy as nfl
import pandas as pd
import duckdb
import polars as pl
pd.set_option('display.max_columns',30)

In [4]:
con = duckdb.connect("data/nfl.duckdb")

In [5]:
end_year = 2026
years=range(2000,end_year)

In [6]:
schedule_raw = nfl.load_schedules(years)
#schedule.to_sql('schedule', conn, if_exists='replace', index=False)
con.execute("CREATE OR REPLACE TABLE schedule AS SELECT * FROM schedule_raw")
schedule_raw

game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,old_game_id,gsis,nfl_detail_id,pfr,pff,espn,ftn,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
str,i64,str,i64,str,str,str,str,i64,str,i64,str,i64,i64,i64,i64,i64,str,str,str,i64,str,i64,i64,str,str,f64,str,str,f64,str,str,i64,str,str,i64,i64,str,str,str,str,str,str,str,str,str
"""2000_01_SF_ATL""",2000,"""REG""",1,"""2000-09-03""","""Sunday""","""13:00""","""SF""",28,"""ATL""",36,"""Home""",8,64,0,2000090310,998,null,"""200009030atl""",null,200903001,null,7,7,null,null,6.5,null,null,46.5,null,null,1,"""dome""","""astroturf""",null,null,"""00-0005755""","""00-0002876""","""Jeff Garcia""","""Chris Chandler""","""Steve Mariucci""","""Dan Reeves""","""Walt Coleman""","""ATL00""","""Georgia Dome"""
"""2000_01_JAX_CLE""",2000,"""REG""",1,"""2000-09-03""","""Sunday""","""13:00""","""JAX""",27,"""CLE""",7,"""Home""",-20,34,0,2000090306,994,null,"""200009030cle""",null,200903005,null,7,7,null,null,-10.5,null,null,38.5,null,null,1,"""outdoors""","""grass""",78,6,"""00-0002110""","""00-0003535""","""Mark Brunell""","""Tim Couch""","""Tom Coughlin""","""Chris Palmer""","""Bernie Kukar""","""CLE00""","""Cleveland Browns Stadium"""
"""2000_01_IND_KC""",2000,"""REG""",1,"""2000-09-03""","""Sunday""","""13:00""","""IND""",27,"""KC""",14,"""Home""",-13,41,0,2000090305,993,null,"""200009030kan""",null,200903012,null,7,7,null,null,-3.5,null,null,44.0,null,null,0,"""outdoors""","""grass""",90,5,"""00-0010346""","""00-0006300""","""Peyton Manning""","""Elvis Grbac""","""Jim Mora""","""Gunther Cunningham""","""Larry Nemmers""","""KAN00""","""Arrowhead Stadium"""
"""2000_01_CHI_MIN""",2000,"""REG""",1,"""2000-09-03""","""Sunday""","""13:00""","""CHI""",27,"""MIN""",30,"""Home""",3,57,0,2000090303,991,null,"""200009030min""",null,200903016,null,7,7,null,null,4.5,null,null,46.5,null,null,1,"""dome""","""astroturf""",null,null,"""00-0011044""","""00-0003739""","""Cade McNown""","""Daunte Culpepper""","""Dick Jauron""","""Dennis Green""","""Ed Hochuli""","""MIN00""","""Hubert H. Humphrey Metrodome"""
"""2000_01_TB_NE""",2000,"""REG""",1,"""2000-09-03""","""Sunday""","""13:00""","""TB""",21,"""NE""",16,"""Home""",-5,37,0,2000090312,1000,null,"""200009030nwe""",null,200903017,null,7,7,null,null,-3.0,null,null,36.0,null,null,0,"""outdoors""","""grass""",71,5,"""00-0009266""","""00-0001361""","""Shaun King""","""Drew Bledsoe""","""Tony Dungy""","""Bill Belichick""","""Jeff Triplette""","""BOS99""","""Foxboro Stadium"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2025_18_DAL_NYG""",2025,"""REG""",18,"""2026-01-04""","""Sunday""","""13:00""","""DAL""",null,"""NYG""",null,"""Home""",null,null,null,2025010411,null,null,"""202501040nyg""","""28684""",401772963,"""7001""",6,6,null,null,null,null,null,null,null,null,1,"""outdoors""","""fieldturf""",null,null,null,null,null,null,"""Brian Schottenheimer""","""Brian Daboll""",null,"""NYC01""","""MetLife Stadium"""
"""2025_18_WAS_PHI""",2025,"""REG""",18,"""2026-01-04""","""Sunday""","""13:00""","""WAS""",null,"""PHI""",null,"""Home""",null,null,null,2025010412,null,null,"""202501040phi""","""28685""",401772959,"""7002""",6,6,null,null,null,null,null,null,null,null,1,"""outdoors""","""grass""",null,null,null,null,null,null,"""Dan Quinn""","""Nick Sirianni""",null,"""PHI00""","""Lincoln Financial Field"""
"""2025_18_BAL_PIT""",2025,"""REG""",18,"""2026-01-04""","""Sunday""","""13:00""","""BAL""",null,"""PIT""",null,"""Home""",null,null,null,2025010413,null,null,"""202501040pit""","""28686""",401772960,"""7003""",10,10,null,null,null,null,null,null,null,null,1,"""outdoors""","""grass""",null,null,null,null,null,null,"""John Harbaugh""","

In [5]:
weekly_raw = nfl.load_player_stats(range(2000,end_year))
weekly_raw.write_parquet('data/weekly.parquet')
con.execute("""CREATE OR REPLACE TABLE weekly AS 
select schedule.game_id, schedule.gameday, weekly_raw.* from weekly_raw
join schedule ON
schedule.season = weekly_raw.season
and schedule.week = weekly_raw.week
and (schedule.home_team = weekly_raw.team
or schedule.away_team = weekly_raw.team)
            """)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
con.execute("select * from weekly order by  season limit 2").df()

,game_id,gameday,player_id,player_name,player_display_name,position,position_group,headshot_url,season,week,season_type,team,opponent_team,completions,attempts,...,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance,fantasy_points,fantasy_points_ppr
0,2000_03_PHI_GB,2000-09-17,00-0009829,D.Levens,Dorsey Levens,RB,RB,https://static.www.nfl.com/image/private/f_aut...,2000,3,REG,GB,PHI,0,0,...,0,0,0,0,0,0,0,NaN,0,0,0,0,0,8.9,10.9
1,2000_05_SEA_KC,2000-10-02,00-0019726,S.Morris,Sylvester Morris,WR,WR,https://static.www.nfl.com/image/private/f_aut...,2000,5,REG,KC,SEA,0,0,...,0,0,0,0,0,0,0,NaN,0,0,0,0,0,4.0,7.0


In [7]:
pbp_raw = nfl.load_pbp(years)

In [8]:
pbp_raw.write_parquet('data/pbp.parquet')

In [9]:
#pbp.write_database('pbp', 'sqlite:///data/nfl.db', if_table_exists='replace')
con.execute("CREATE OR REPLACE TABLE pbp AS SELECT * FROM pbp_raw")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [7]:
rosters_raw = nfl.load_rosters(range(2002,end_year))
con.execute("CREATE OR REPLACE TABLE rosters AS SELECT * FROM rosters_raw")

In [8]:
ids_raw = nfl.load_ff_playerids()
con.execute("CREATE OR REPLACE TABLE ids AS SELECT * FROM ids_raw")

In [9]:
snaps_raw = nfl.load_snap_counts(range(2012,end_year))
team_map = {'OAK':'LV',
            'STL':'LA',
            'SD':'LAC'}
snaps_raw = snaps_raw.to_pandas()  # temporarily convert
snaps_raw['team'] = snaps_raw['team'].map(lambda x: team_map.get(x,x))
snaps_raw = pl.from_pandas(snaps_raw)
#snaps.to_sql('snaps', conn, if_exists='replace', index=False)
con.execute("CREATE OR REPLACE TABLE snaps AS SELECT * FROM snaps_raw")

In [19]:
depth_result = nfl.load_depth_charts(range(2001,end_year)).unique()
con.register("depth_result", depth_result)
con.execute("""drop table if exists depth""")
con.execute("""
create or replace table depth as (
with current as (
	select s.gameday, s.game_id,
	ROW_NUMBER() OVER (PARTITION BY d.team, d.gsis_id, d.pos_grp, s.gameday ORDER BY d.dt DESC) as row_num,
	d.player_name, d.team, d.gsis_id, d.pos_abb, d.pos_rank, dt, 
	case when d.pos_grp = 'Base 4-3 D' then 'Defense'
		when d.pos_grp = 'Base 3-4 D' then 'Defense'
		when d.pos_grp = '3WR 1TE' then 'Offense'
		else d.pos_grp end as formation
	from depth_result as d
	join schedule as s on (s.home_team = d.team
	or s.away_team = d.team)
	and d.dt <= s.gameday
	and cast(d.dt as date ) >= cast(s.gameday as date) - INTERVAL 5 DAY
	where dt > '2025-07-01'
),
pre2025 as (
	select s.gameday, s.game_id,
	d.football_name as player_name, d.club_code as team, d.gsis_id,
	 case when d.depth_position ilike  '%\n%' then d.position else d.depth_position end as pos_abb,
 d.depth_team as pos_rank, d.formation
	from depth_result as d
	join schedule as s on (s.home_team = d.club_code
	or s.away_team = d.club_code)
	and d.season = s.season
	and d.week = s.week
	and d.game_type = s.game_type
),
combined as (
select gameday, game_id, player_name, team, gsis_id, case when pos_abb = 'PK' then 'K' else pos_abb end as position, pos_rank, formation
from current 
where row_num = 1
and (formation = 'Offense' or position = 'K')
union
select gameday, game_id, player_name, team, gsis_id, case when pos_abb = 'PK' then 'K' else pos_abb end as position, pos_rank, formation
from pre2025
)
select *,SPLIT_PART(game_id, '_', 1)::int as season,
        SPLIT_PART(game_id, '_', 2)::int as week,
                     ROW_NUMBER() OVER (
           PARTITION BY team, game_id, position, formation
           ORDER BY game_id, team, pos_rank
       ) - 1 AS dense_depth
from combined
)
""").fetchdf()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,775599


In [21]:
con.execute("select * from depth where game_id ilike '2025_04%' and position = 'K' ").df()

,gameday,game_id,player_name,team,gsis_id,position,pos_rank,formation,season,week,dense_depth
0,2025-09-29,2025_04_CIN_DEN,Wil Lutz,DEN,00-0032569,K,1,Special Teams,2025,4,0
1,2025-09-28,2025_04_PHI_TB,Chase McLaughlin,TB,00-0035358,K,1,Special Teams,2025,4,0
2,2025-09-28,2025_04_CAR_NE,Andy Borregales,NE,00-0040200,K,1,Special Teams,2025,4,0
3,2025-09-28,2025_04_BAL_KC,Tyler Loop,BAL,00-0040074,K,1,Special Teams,2025,4,0
4,2025-09-28,2025_04_MIN_PIT,Chris Boswell,PIT,00-0031136,K,1,Special Teams,2025,4,0
5,2025-09-28,2025_04_NO_BUF,Matt Prater,BUF,00-0023853,K,1,Special Teams,2025,4,0
6,2025-09-28,2025_04_NO_BUF,Tyler Bass,BUF,00-0036162,K,2,Special Teams,2025,4,1
7,2025-09-28,2025_04_CHI_LV,Daniel Carlson,LV,00-0034161,K,1,Special Teams,2025,4,0
8,2025-09-28,2025_04_JAX_SF,Eddy Pineiro,SF,00-0034173,K,1,Special Teams,2025,4,0
9,2025-09-28,2025_04_LAC_NYG,Graham Gano,NYG,00-0026858,K,1,Special Teams,2025,4,0


In [22]:
con.execute("select * from depth where game_id ilike '2025_04%' and position = 'K' ").df()

,gameday,game_id,player_name,team,gsis_id,position,pos_rank,formation,season,week,dense_depth
0,2025-09-29,2025_04_CIN_DEN,Wil Lutz,DEN,00-0032569,K,1,Special Teams,2025,4,0
1,2025-09-28,2025_04_PHI_TB,Chase McLaughlin,TB,00-0035358,K,1,Special Teams,2025,4,0
2,2025-09-28,2025_04_CAR_NE,Andy Borregales,NE,00-0040200,K,1,Special Teams,2025,4,0
3,2025-09-28,2025_04_BAL_KC,Tyler Loop,BAL,00-0040074,K,1,Special Teams,2025,4,0
4,2025-09-28,2025_04_MIN_PIT,Chris Boswell,PIT,00-0031136,K,1,Special Teams,2025,4,0
5,2025-09-28,2025_04_NO_BUF,Matt Prater,BUF,00-0023853,K,1,Special Teams,2025,4,0
6,2025-09-28,2025_04_NO_BUF,Tyler Bass,BUF,00-0036162,K,2,Special Teams,2025,4,1
7,2025-09-28,2025_04_CHI_LV,Daniel Carlson,LV,00-0034161,K,1,Special Teams,2025,4,0
8,2025-09-28,2025_04_JAX_SF,Eddy Pineiro,SF,00-0034173,K,1,Special Teams,2025,4,0
9,2025-09-28,2025_04_LAC_NYG,Graham Gano,NYG,00-0026858,K,1,Special Teams,2025,4,0


In [23]:
con.execute("select gameday, * from schedule where home_team = 'MIA'").df().tail(3)

,gameday,game_id,season,game_type,week,gameday_1,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,...,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
211,2025-11-30,2025_13_NO_MIA,2025,REG,13,2025-11-30,Sunday,13:00,NO,17.0,MIA,21.0,Home,4.0,38.0,...,-115,0,outdoors,grass,76.0,10.0,00-0040743,00-0036212,Tyler Shough,Tua Tagovailoa,Kellen Moore,Mike McDaniel,Alan Eck,MIA00,Hard Rock Stadium
212,2025-12-21,2025_16_CIN_MIA,2025,REG,16,2025-12-21,Sunday,20:20,CIN,NaN,MIA,NaN,Home,NaN,NaN,...,None,0,outdoors,grass,NaN,NaN,None,None,None,None,Zac Taylor,Mike McDaniel,None,MIA00,Hard Rock Stadium
213,2025-12-28,2025_17_TB_MIA,2025,REG,17,2025-12-28,Sunday,13:00,TB,NaN,MIA,NaN,Home,NaN,NaN,...,None,0,outdoors,grass,NaN,NaN,None,None,None,None,Todd Bowles,Mike McDaniel,None,MIA00,Hard Rock Stadium


In [24]:
con.close()

In [28]:
dep = depth_result.to_pandas()
dep.loc[(dep.season==2025) &
                  (dep.week==4) &
                  (dep.club_code=='BUF')
                  & (dep.position=='K')]

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,dt,team,player_name,espn_id,pos_grp_id,pos_grp,pos_id,pos_name,pos_abb,pos_slot,pos_rank
